<a href="https://colab.research.google.com/github/LC1332/3D-Mantis-Vision-Dissertation/blob/main/%E6%89%B9%E9%87%8F%E8%AF%AD%E9%9F%B3%E5%90%88%E6%88%90%E7%81%AB%E5%B1%B1%E7%89%88%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

# 文件路径
file_path = '/content/drive/MyDrive/shiwan/output/10kqs_with_NER_fix.jsonl'

# 初始化一个空列表，用于存储字典
datas = []

# 打开文件并读取每一行
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # 将每行的JSON字符串转换为字典
        datas.append(json.loads(line))


In [ ]:
import re

def extract_keyword(json_text, keyword):
    # 正则表达式模式，用于匹配关键词及其值
    pattern = fr'"\s*{keyword}\s*"\s*:\s*"\s*([^"]+)\s*"\s*'

    # 使用正则表达式搜索匹配项
    match = re.search(pattern, json_text)

    # 如果找到匹配项，返回匹配的值；否则返回 None
    return match.group(1) if match else None


def get_major_word( data ):
    NER_response_str = data["NER_response"]
    # 解析字符串为字典
    NER_response_str = NER_response_str.replace("(", "[").replace(")", "]")

    question = data["question"]  # 问题

    try:
        NER_response = json.loads(NER_response_str)
        major_keyword = NER_response["关键实体"]  # 自动抽取的关键字
    except:
        major_keyword = extract_keyword(NER_response_str, "关键实体")
        NER_response = None
        if major_keyword is None:
            major_keyword = data['question']

    return major_keyword

In [ ]:

def remove_punctuation(input_string):
    # 定义需要去除的标点符号
    punctuation = r"""[!\"#$%&\'()*+,\-./:;<=>?@\[\\\]^_`{|}~]"""
    # 使用 re.sub() 替换掉所有的标点符号
    output_string = re.sub(punctuation, '', input_string)
    output_string = output_string.replace("，", "_")
    return output_string

In [ ]:
import base64
import json
import uuid
import requests
from google.colab import userdata

class VolcanoTTS:
    def __init__(self, voice_type = None, speed = None):
        # 填写平台申请的appid, access_token以及cluster
        self.appid = userdata.get('volcano_app_id')
        self.access_token = userdata.get('volcano_access_token')
        self.cluster = "volcano_tts"
        if voice_type is None:
            self.voice_type = "BV426_streaming"
        else:
            self.voice_type = voice_type
        if speed is None:
            self.speed = 0.9
        else:
            self.speed = speed
        self.host = "openspeech.bytedance.com"
        self.api_url = f"https://{self.host}/api/v1/tts"
        self.header = {"Authorization": f"Bearer;{self.access_token}"}

    def text2audio(self, content, save_name):
        request_json = {
            "app": {
                "appid": self.appid,
                "token": "access_token",
                "cluster": self.cluster
            },
            "user": {
                "uid": "388808087185088"
            },
            "audio": {
                "voice_type": self.voice_type,
                "encoding": "mp3",
                "speed_ratio": self.speed,
                "volume_ratio": 1.0,
                "pitch_ratio": 1.0,
            },
            "request": {
                "reqid": str(uuid.uuid4()),
                "text": content,
                "text_type": "plain",
                "operation": "query",
                "with_frontend": 1,
                "frontend_type": "unitTson"
            }
        }

        try:
            resp = requests.post(self.api_url, json.dumps(request_json), headers=self.header, timeout = 40)
            # print(f"resp body: \n{resp.json()}")
            if "data" in resp.json():
                data = resp.json()["data"]
                with open(save_name, "wb") as file_to_save:
                    file_to_save.write(base64.b64decode(data))
        except Exception as e:
            print(f"An error occurred: {e}")
            raise

# 使用示例
# tts = VolcanoTTS(voice_type="BV700_streaming")
# tts.text2audio(content="字节跳动语音合成", save_name="test_submit.mp3")


In [ ]:
import os
import shutil
from tqdm import tqdm
import zipfile

save_step = 50
target_folder = '/content/drive/MyDrive/shiwan/audios_volc'

def check_and_create_directory(path):
    """确保目录存在，如果不存在则创建"""
    if not os.path.exists(path):
        os.makedirs(path)

check_and_create_directory(target_folder)

def zip_directory(source_dir, output_filename):
    """将指定目录压缩成zip文件"""
    with zipfile.ZipFile(output_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(source_dir):
            for file in files:
                zipf.write(os.path.join(root, file), arcname=os.path.relpath(os.path.join(root, file), os.path.join(source_dir, '..')))

tts = VolcanoTTS(voice_type="BV426_streaming")

def text2audio(content, save_name):
    tts.text2audio(content, save_name)

for data in tqdm(datas):
    id = data['id']
    batch_id = id // save_step

    zip_file_path = os.path.join(target_folder, f'audio_in_batch_{batch_id}.zip')
    batch_folder_path = f'audio_batch_{batch_id}'

    # 检查是否存在zip文件
    if os.path.exists(zip_file_path):
        continue

    # 确保批次文件夹存在
    check_and_create_directory(batch_folder_path)

    # 准备文件名和文本
    question_text = data['question'] + "?"
    short_answer = data['summary']
    question_no_punc = remove_punctuation(data['question'])
    category = data['category']
    save_name_prefix = f"{id}_{category}_{question_no_punc}"
    question_save_name = os.path.join(batch_folder_path, f"{save_name_prefix}_question.mp3")
    short_answer_save_name = os.path.join(batch_folder_path, f"{save_name_prefix}_short_answer.mp3")

    for _ in range(3):
        # 生成音频文件
        if not os.path.exists(question_save_name):
            try:
                text2audio(question_text, question_save_name)
            except:
                pass
        if not os.path.exists(short_answer_save_name):
            try:
                text2audio(short_answer, short_answer_save_name)
            except:
                pass

    # 如果是批次的最后一个文件
    if (id + 1) % save_step == 0 or id == len(datas) - 1:
        # 压缩文件夹
        zip_directory(batch_folder_path, zip_file_path)
        # 清理临时文件夹
        shutil.rmtree(batch_folder_path)
        print(f"Batch {batch_id} processed and moved to {zip_file_path}")

print("All data processed.")


  9%|▉         | 278/3107 [00:19<00:38, 73.83it/s]

An error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


  9%|▉         | 282/3107 [00:36<09:16,  5.07it/s]

An error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
An error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


  9%|▉         | 285/3107 [01:10<21:02,  2.24it/s]

...

Batch 61 processed and moved to /content/drive/MyDrive/shiwan/audios_volc/audio_in_batch_61.zip


100%|█████████▉| 3102/3107 [7:20:26<00:44,  8.88s/it]

An error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


100%|██████████| 3107/3107 [7:21:38<00:00,  8.53s/it]

Batch 62 processed and moved to /content/drive/MyDrive/shiwan/audios_volc/audio_in_batch_62.zip
All data processed.


In [ ]:
from IPython.display import Audio

# 替换'url_to_mp3'为你的MP3文件的URL
Audio(question_save_name, autoplay=True)


In [ ]:
from IPython.display import Audio

# 替换'url_to_mp3'为你的MP3文件的URL
Audio(short_answer_save_name, autoplay=True)


---

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/shiwan/audios_volc

audio_in_batch_0.zip   audio_in_batch_24.zip  audio_in_batch_39.zip  audio_in_batch_53.zip
audio_in_batch_10.zip  audio_in_batch_25.zip  audio_in_batch_3.zip   audio_in_batch_54.zip
audio_in_batch_11.zip  audio_in_batch_26.zip  audio_in_batch_40.zip  audio_in_batch_55.zip
audio_in_batch_12.zip  audio_in_batch_27.zip  audio_in_batch_41.zip  audio_in_batch_56.zip
audio_in_batch_13.zip  audio_in_batch_28.zip  audio_in_batch_42.zip  audio_in_batch_57.zip
audio_in_batch_14.zip  audio_in_batch_29.zip  audio_in_batch_43.zip  audio_in_batch_58.zip
audio_in_batch_15.zip  audio_in_batch_2.zip   audio_in_batch_44.zip  audio_in_batch_59.zip
audio_in_batch_16.zip  audio_in_batch_30.zip  audio_in_batch_45.zip  audio_in_batch_5.zip
audio_in_batch_17.zip  audio_in_batch_31.zip  audio_in_batch_46.zip  audio_in_batch_60.zip
audio_in_batch_18.zip  audio_in_batch_32.zip  audio_in_batch_47.zip  audio_in_batch_61.zip
audio_in_batch_19.zip  audio_in_batch_33.zip  audio_in_batch_48.zip  audio_in_batch_62.zip


请帮我实现一段python代码，把

/content/drive/MyDrive/shiwan/audios_volc

目录下的所有zip 转移到到当前目录下

再把这些文件压缩成一个完整的zip all_audios.zip

In [ ]:
import os
import shutil
import zipfile

# 源目录和目标目录
source_dir = '/content/drive/MyDrive/shiwan/audios_volc'
target_dir = '.'

# 遍历源目录中的所有文件
for filename in os.listdir(source_dir):
    # 检查是否是zip文件
    if filename.endswith('.zip'):
        source_file = os.path.join(source_dir, filename)
        target_file = os.path.join(target_dir, filename)

        # 移动zip文件到当前目录
        shutil.copy(source_file, target_file)
        print(f'Moved {filename} to current directory.')




Moved audio_in_batch_0.zip to current directory.
Moved audio_in_batch_1.zip to current directory.
Moved audio_in_batch_2.zip to current directory.
Moved audio_in_batch_3.zip to current directory.
Moved audio_in_batch_4.zip to current directory.
Moved audio_in_batch_5.zip to current directory.
Moved audio_in_batch_6.zip to current directory.
Moved audio_in_batch_7.zip to current directory.
Moved audio_in_batch_8.zip to current directory.
Moved audio_in_batch_9.zip to current directory.
Moved audio_in_batch_10.zip to current directory.
Moved audio_in_batch_11.zip to current directory.
Moved audio_in_batch_12.zip to current directory.
Moved audio_in_batch_13.zip to current directory.
Moved audio_in_batch_14.zip to current directory.
Moved audio_in_batch_15.zip to current directory.
Moved audio_in_batch_16.zip to current directory.
Moved audio_in_batch_17.zip to current directory.
Moved audio_in_batch_18.zip to current directory.
Moved audio_in_batch_19.zip to current directory.
Moved audi

RuntimeError: File size unexpectedly exceeded ZIP64 limit

In [ ]:
# 创建一个新的zip文件，包含当前目录下的所有zip文件
with zipfile.ZipFile('all_audios.zip', 'w') as zipf:
    for filename in os.listdir(target_dir):
        if filename.endswith('.zip') and filename.startswith('audio_in_batch'):
            # 添加文件到zip
            zipf.write(filename)
            print(f'Added {filename} to all_audios.zip')

print('Created all_audios.zip with all zip files in the current directory.')

Added audio_in_batch_45.zip to all_audios.zip
Added audio_in_batch_3.zip to all_audios.zip
Added audio_in_batch_57.zip to all_audios.zip
Added audio_in_batch_32.zip to all_audios.zip
Added audio_in_batch_55.zip to all_audios.zip
Added audio_in_batch_39.zip to all_audios.zip
Added audio_in_batch_47.zip to all_audios.zip
Added audio_in_batch_41.zip to all_audios.zip
Added audio_in_batch_25.zip to all_audios.zip
Added audio_in_batch_61.zip to all_audios.zip
Added audio_in_batch_1.zip to all_audios.zip
Added audio_in_batch_28.zip to all_audios.zip
Added audio_in_batch_51.zip to all_audios.zip
Added audio_in_batch_15.zip to all_audios.zip
Added audio_in_batch_10.zip to all_audios.zip
Added audio_in_batch_17.zip to all_audios.zip
Added audio_in_batch_12.zip to all_audios.zip
Added audio_in_batch_19.zip to all_audios.zip
Added audio_in_batch_34.zip to all_audios.zip
Added audio_in_batch_24.zip to all_audios.zip
Added audio_in_batch_0.zip to all_audios.zip
Added audio_in_batch_35.zip to all_au

In [ ]:
import os
import shutil
import zipfile

# 源目录和目标目录
source_dir = '/content/drive/MyDrive/shiwan/audios_volc'
target_dir = '.'

# 文件列表，用于存储需要加入到压缩文件中的文件名
zip_files = []

# 遍历源目录中的所有文件
for filename in os.listdir(source_dir):
    # 检查是否是zip文件
    if filename.endswith('.zip'):
        source_file = os.path.join(source_dir, filename)
        target_file = os.path.join(target_dir, filename)

        # 复制zip文件到当前目录
        shutil.copy(source_file, target_file)
        print(f'Copied {filename} to current directory.')

        # 添加到文件列表
        zip_files.append(filename)

# 创建一个新的zip文件，包含当前目录下的所有zip文件
with zipfile.ZipFile('all_audios.zip', 'w') as zipf:
    for filename in zip_files:
        # 添加文件到zip
        zipf.write(filename)
        print(f'Added {filename} to all_audios.zip')

# 拷贝all_audios.zip回原始目录
destination_path = os.path.join(source_dir, 'all_audios.zip')
shutil.copy('all_audios.zip', destination_path)
print(f'Copied all_audios.zip back to {source_dir}')

print('Completed all tasks.')


Copied audio_in_batch_0.zip to current directory.
Copied audio_in_batch_1.zip to current directory.
Copied audio_in_batch_2.zip to current directory.
Copied audio_in_batch_3.zip to current directory.
Copied audio_in_batch_4.zip to current directory.
Copied audio_in_batch_5.zip to current directory.
Copied audio_in_batch_6.zip to current directory.
Copied audio_in_batch_7.zip to current directory.
Copied audio_in_batch_8.zip to current directory.
Copied audio_in_batch_9.zip to current directory.
Copied audio_in_batch_10.zip to current directory.
Copied audio_in_batch_11.zip to current directory.
Copied audio_in_batch_12.zip to current directory.
Copied audio_in_batch_13.zip to current directory.
Copied audio_in_batch_14.zip to current directory.
Copied audio_in_batch_15.zip to current directory.
Copied audio_in_batch_16.zip to current directory.
Copied audio_in_batch_17.zip to current directory.
Copied audio_in_batch_18.zip to current directory.
Copied audio_in_batch_19.zip to current d

KeyboardInterrupt: 

In [ ]:
# 拷贝all_audios.zip回原始目录
destination_path = os.path.join(source_dir, 'all_audios.zip')
shutil.copy('all_audios.zip', destination_path)
print(f'Copied all_audios.zip back to {source_dir}')

print('Completed all tasks.')

Copied all_audios.zip back to /content/drive/MyDrive/shiwan/audios_volc
Completed all tasks.
